In [1]:
# importar las librerías requeridas
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import plotly.graph_objs as go
from datetime import datetime,timedelta
import time


#Importar librería de google drive para la carga de los archivos de datos requeridos.
from google.colab import drive
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

# Carga del Dataset

In [2]:

drive.mount('/content/drive')

# Se crea un path específico para facilitar la lectura de los archivos posteriormente desde google drive
path = "/content/drive/MyDrive/seminario-master/Datasets_salida/"

dataset=pd.read_csv(path + 'dataset_final/dataset_demada_Antioquia.csv', sep=',')

#Normalizar datos negativos
dataset.loc[dataset['ALLSKY_SFC_SW_DWN_pond'] < 0, 'ALLSKY_SFC_SW_DWN_pond'] = 0
dataset.loc[dataset['ALLSKY_SFC_SW_DWN_BAJO CAUCA'] < 0, 'ALLSKY_SFC_SW_DWN_BAJO CAUCA'] = 0
dataset.loc[dataset['ALLSKY_SFC_SW_DWN_URABA'] < 0, 'ALLSKY_SFC_SW_DWN_URABA'] = 0

dataset.head()

Mounted at /content/drive


,TIPO DIA,AÑO,MES,DIA,HORA,DEMANDA,Demanda_anterior,Total Residencial,Total No Residencial,Industrial,...,T2M_MAGDALENA MEDIO,T2M_NORDESTE,T2M_NORTE,T2M_OCCIDENTE,T2M_ORIENTE,T2M_SUROESTE,T2M_URABA,Date,PRECIO BOLSA,FECHA
0,1--ENE,2017,1,1,0,629.88,629.88,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-01
1,1--ENE,2017,1,1,1,583.18,629.88,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-01
2,1--ENE,2017,1,1,2,552.80,583.18,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-01
3,1--ENE,2017,1,1,3,530.52,552.80,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-01
4,1--ENE,2017,1,1,4,516.12,530.52,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-01


In [3]:
dataset.describe()

,AÑO,MES,DIA,HORA,DEMANDA,Demanda_anterior,Total Residencial,Total No Residencial,Industrial,total_usuarios,...,T2M_AREA METROPOLITANA DEL VALLE DE ABURRÁ,T2M_BAJO CAUCA,T2M_MAGDALENA MEDIO,T2M_NORDESTE,T2M_NORTE,T2M_OCCIDENTE,T2M_ORIENTE,T2M_SUROESTE,T2M_URABA,PRECIO BOLSA
count,64392.000000,64392.000000,64392.000000,64392.00000,64392.000000,64392.000000,5.533900e+04,55339.000000,55339.000000,5.533900e+04,...,55339.000000,55339.000000,55339.000000,55339.000000,55339.000000,55339.000000,55339.000000,55339.000000,55339.000000,52584.000000
mean,2020.189340,6.339173,15.692881,11.50000,1072.547853,1072.542634,2.312467e+06,243618.800484,24015.541300,2.556085e+06,...,16.867350,24.960667,23.203698,21.143583,18.141474,17.942849,19.450958,18.220056,25.441264,253.260902
std,2.128556,3.476767,8.807676,6.92224,228.108647,228.114931,2.626740e+05,38811.456224,3060.981072,2.965981e+05,...,21.332014,21.521335,21.495741,21.443288,21.355576,21.338526,21.403966,21.346496,21.441409,210.161998
min,2017.000000,1.000000,1.000000,0.00000,434.555730,434.555730,0.000000e+00,0.000000,0.000000,0.000000e+00,...,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,61.458540
25%,2018.000000,3.000000,8.000000,5.75000,881.955582,881.915000,2.237105e+06,211689.000000,22648.000000,2.448794e+06,...,15.215000,23.206667,21.380000,19.396364,16.491667,16.373158,17.668500,16.642273,24.269091,109.483290
50%,2020.000000,6.000000,16.000000,11.50000,1080.213600,1080.213600,2.348937e+06,264567.000000,24463.000000,2.595041e+06,...,16.665000,24.646667,22.855000,20.819091,17.892000,17.753684,19.105000,18.023182,25.366364,185.719680
75%,2022.000000,9.000000,23.000000,17.25000,1266.160660,1266.160660,2.430212e+06,272412.000000,25767.000000,2.702624e+06,...,19.290000,27.226667,25.521667,23.571818,20.573000,20.279474,21.988750,20.536136,27.356364,309.780010
max,2024.000000,12.000000,31.000000,23.00000,1619.206180,1619.206180,2.539229e+06,281148.000000,27292.000000,2.820377e+06,...,26.755000,38.510000,35.958333,33.237273,28.708000,26.809474,29.944500,26.953182,33.801818,1595.609770


## Clasificación por tipo día

*   Elemento de la lista
*   Elemento de la lista



In [4]:


# Crear un DataFrame de ejemplo
data = {
    'TIPO DIA': ['1--ENE', '2--ENE', 'MIVENE', 'JUVENE', 'VIVENE', 'SAALFENE',
       'DOALFENE', 'LFENE', 'MAVENE', 'SAVENE', 'DOVENE', 'LUNES',
       'MARTES', 'MIERCOLES', 'JUEVES', 'VIERNES', 'SABADO', 'DOMINGO',
       'SAALF', 'DOALF', 'LF', 'MADLF', 'MSS', 'JSS', 'VSS', 'SSS', 'DSS',
       '1--MAY', '20--JUL', '7--AGO', '8--DIC', 'VIVDIC', 'SAVDIC',
       'DOVDIC', '24--DIC', '25--DIC', 'MIVDIC', 'JUVDIC', '31--DIC',
       'LUVDIC', 'MAVDIC', 'LUVENE'],
}
df = pd.DataFrame(data)

# Definir una función para mapear los valores de 'TIPO DIA' a 'T_DIA'
def map_tipo_dia(tipo_dia):
    if tipo_dia in ['MARTES', 'MIERCOLES', 'JUEVES', 'VIERNES', 'MSS']:
        return 'MAR-VIE'
    elif tipo_dia in ['DOMINGO', 'DOALF', 'DOALFENE', 'DOVDIC', 'DOVENE', 'DSS', 'JSS', 'VSS','25--DIC', '1--ENE']:
        return 'DOM'
    elif tipo_dia in ['SABADO','SÁBADO', 'SAALF','SSS', 'SAALFENE', 'SAVDIC', 'SAVENE', '24--DIC', '31--DIC']:
        return 'SAB'
    elif tipo_dia in ['LF', 'LFENE']:
        return 'LFEST'
    elif tipo_dia in ['1--MAY', '20--JUL', '7--AGO', '8--DIC']:
        return 'OFEST'
    #elif tipo_dia in ['25--DIC', '1--ENE']:
        #return '25-1'
    elif tipo_dia in ['MAVDIC' , 'MAVENE', 'MIVDIC', 'MIVENE', 'JUVDIC','JUVENE','VIVDIC','VIVENE','2--ENE']:
        return 'MV-DE'
    elif tipo_dia in ['LUVDIC', 'LUVENE']:
        return 'L-DE'
    elif tipo_dia in ['LUNES', 'MADLF']:
        return 'LUN-MADLF'
    else:
        return tipo_dia

# Aplicar la función a la columna 'TIPO DIA' para crear la nueva columna 'T_DIA'
dataset['T_DIA'] = dataset['TIPO DIA'].apply(map_tipo_dia)



In [5]:
dataset = dataset.sort_values(by=['AÑO', 'MES', 'DIA', 'HORA'], ascending=True)
# Reiniciar el índice
dataset = dataset.reset_index(drop=True)


# Crear una nueva columna llamada 'Demanda_anterior'
dataset['Demanda_24h_antes'] = dataset['DEMANDA'].shift(24)
dataset['Demanda_25h_antes'] = dataset['Demanda_anterior'].shift(24)


# Llenar el primer valor de 'Demanda_anterior' con el mismo valor de DEMANDA
#dataset.at[0, 'Demanda_24h_antes'] = dataset.at[0, 'DEMANDA']
#dataset.at[0, 'Demanda_25h_antes'] = dataset.at[0, 'DEMANDA']

# Llenar el primer valor de 'Demanda_anterior' con el mismo valor de DEMANDA
# Iterar sobre los primeros 24 registros
for i in range(24):
    # Establecer el valor de 'Demanda_25h_antes' igual al valor de 'DEMANDA' en el registro actual
    dataset.at[i, 'Demanda_25h_antes'] = dataset.at[i, 'Demanda_anterior']
    dataset.at[i, 'Demanda_24h_antes'] = dataset.at[i, 'DEMANDA']

dataset.at[24, 'Demanda_25h_antes'] = dataset.at[24, 'Demanda_anterior']





In [6]:
dataset.tail()

,TIPO DIA,AÑO,MES,DIA,HORA,DEMANDA,Demanda_anterior,Total Residencial,Total No Residencial,Industrial,...,T2M_OCCIDENTE,T2M_ORIENTE,T2M_SUROESTE,T2M_URABA,Date,PRECIO BOLSA,FECHA,T_DIA,Demanda_24h_antes,Demanda_25h_antes
64387,DOMINGO,2024,5,5,19,1203.31046,1172.25029,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-05,DOM,1262.81191,1266.29320
64388,DOMINGO,2024,5,5,20,1168.28671,1203.31046,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-05,DOM,1193.95431,1262.81191
64389,DOMINGO,2024,5,5,21,1112.48057,1168.28671,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-05,DOM,1104.82068,1193.95431
64390,DOMINGO,2024,5,5,22,1044.26845,1112.48057,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-05,DOM,1016.67490,1104.82068
64391,DOMINGO,2024,5,5,23,965.97154,1044.26845,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-05,DOM,934.64735,1016.67490


In [7]:

dataset['Date'] = pd.to_datetime(dataset['Date'])

# Obtener los valores únicos de T_DIA
unique_t_dia = dataset['T_DIA'].unique()
# Crear un diccionario para almacenar los DataFrames filtrados
filtered_datasets = {}
filtered_datasets_list = []


# Iterar sobre los diferentes valores de T_DIA
for t_dia in unique_t_dia:
    # Filtrar los datos para el valor actual de T_DIA
    filtered_demand = dataset[dataset['T_DIA'] == t_dia]
    filtered_demand.loc[:, 'Date'] = pd.to_datetime(filtered_demand['Date'])
    filtered_demand = filtered_demand.reset_index(drop=True)

    # Crear una nueva columna llamada 'Delta_24h'
    filtered_demand['Delta_24h'] = filtered_demand['DEMANDA'].shift(24)
    # Crear una nueva columna llamada 'Delta_25h'
    filtered_demand['Delta_25h'] = filtered_demand['Demanda_anterior'].shift(24)



    # Establecer el valor de 'Delta_24h' y 'Delta_25h' para los primeros 24 registros
    for i in range(24):
        filtered_demand.at[i, 'Delta_24h'] = filtered_demand.at[i, 'DEMANDA']
        filtered_demand.at[i, 'Delta_25h'] = filtered_demand.at[i, 'Demanda_anterior']

    # Establecer el valor de 'Delta_25h' para el registro 25
    filtered_demand.at[24, 'Delta_25h'] = filtered_demand.at[24, 'Demanda_anterior']



    # Normalizar las características numéricas
    #scaler = StandardScaler()
    #filtered_demand[numeric_columns] = scaler.fit_transform(filtered_demand[numeric_columns])

    # Almacenar el DataFrame filtrado en el diccionario
    filtered_datasets[t_dia] = filtered_demand
    filtered_datasets_list.append(filtered_demand)


# Concatenar los DataFrames en un solo conjunto de datos
final_dataset = pd.concat(filtered_datasets_list, ignore_index=True)

final_dataset = final_dataset.sort_values(by=['AÑO', 'MES', 'DIA', 'HORA'], ascending=True)


#sab_dataset = filtered_datasets['SAB']


In [8]:
final_dataset.tail()

,TIPO DIA,AÑO,MES,DIA,HORA,DEMANDA,Demanda_anterior,Total Residencial,Total No Residencial,Industrial,...,T2M_SUROESTE,T2M_URABA,Date,PRECIO BOLSA,FECHA,T_DIA,Demanda_24h_antes,Demanda_25h_antes,Delta_24h,Delta_25h
9691,DOMINGO,2024,5,5,19,1203.31046,1172.25029,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,2024-05-05,DOM,1262.81191,1266.29320,1174.32912,1157.92933
9692,DOMINGO,2024,5,5,20,1168.28671,1203.31046,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,2024-05-05,DOM,1193.95431,1262.81191,1144.15536,1174.32912
9693,DOMINGO,2024,5,5,21,1112.48057,1168.28671,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,2024-05-05,DOM,1104.82068,1193.95431,1072.46013,1144.15536
9694,DOMINGO,2024,5,5,22,1044.26845,1112.48057,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,2024-05-05,DOM,1016.67490,1104.82068,1008.73042,1072.46013
9695,DOMINGO,2024,5,5,23,965.97154,1044.26845,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,2024-05-05,DOM,934.64735,1016.67490,932.82516,1008.73042


In [9]:
def metricas(y_test, y_pred):

  r2 = r2_score(y_test, y_pred)
  mse = mean_squared_error(y_true=y_test, y_pred=y_pred, squared=True)
  rmse = mean_squared_error(y_true=y_test, y_pred=y_pred, squared=False)
  mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
  mape=mean_absolute_percentage_error(y_true=y_test, y_pred=y_pred)
  print(f"r2:{r2}")
  print(f"MSE: {mse}")
  print(f"RMSE: {rmse}")
  print(f"MAE: {mae}")
  print(f"MAPE: {mape}")

  # Calcular la diferencia absoluta entre los valores de Y_test y Y_pred
  diferencia_absoluta = np.abs(y_test - y_pred)

  # Calcular el porcentaje de desviación
  porcentaje_desviacion = (diferencia_absoluta / y_test) * 100



  print('shapes:', y_test.shape, y_pred.shape, porcentaje_desviacion.shape)

  # Crear el DataFrame
  results= pd.DataFrame({'y_test': y_test,
                    'y_pred': y_pred,
                    '%error': porcentaje_desviacion})

  # Imprimir el DataFrame
  print(results.head(50))
  return results,rmse

In [10]:
fecha_inicio_pred='2024-01-01'
#fecha_fin_pred='2024-03-31'
semanas=13
dias_to_pred=14
fecha_inicio_pred= datetime.strptime(fecha_inicio_pred, "%Y-%m-%d")
fecha_inicio_pred= fecha_inicio_pred - timedelta(days=dias_to_pred//2)
fecha_fin_pred=fecha_inicio_pred + timedelta(days=dias_to_pred-1)

In [11]:
fecha_inicio_pred

datetime.datetime(2023, 12, 25, 0, 0)

In [12]:
filtered=filtered_datasets['MAR-VIE'][filtered_datasets['MAR-VIE']['HORA'] == 12]
filtered = filtered[(filtered['AÑO'] >= 2018) & (filtered['AÑO'] != 2020)]
filtered[filtered['FECHA'] < fecha_inicio_pred.strftime("%Y-%m-%d")]

,TIPO DIA,AÑO,MES,DIA,HORA,DEMANDA,Demanda_anterior,Total Residencial,Total No Residencial,Industrial,...,T2M_SUROESTE,T2M_URABA,Date,PRECIO BOLSA,FECHA,T_DIA,Demanda_24h_antes,Demanda_25h_antes,Delta_24h,Delta_25h
4260,MARTES,2018,1,16,12,1224.50000,1257.94000,2131430.0,203894.0,20905.0,...,20.282273,27.353636,2018-01-16,127.52105,2018-01-16,MAR-VIE,1218.27000,1240.09000,1266.46000,1286.74000
4284,MIERCOLES,2018,1,17,12,1240.38000,1256.32000,2131430.0,203894.0,20905.0,...,20.876818,26.397273,2018-01-17,125.58802,2018-01-17,MAR-VIE,1224.50000,1257.94000,1224.50000,1257.94000
4308,JUEVES,2018,1,18,12,1337.90000,1328.47000,2131430.0,203894.0,20905.0,...,21.221364,26.263636,2018-01-18,128.47991,2018-01-18,MAR-VIE,1240.38000,1256.32000,1240.38000,1256.32000
4332,VIERNES,2018,1,19,12,1275.87000,1298.23000,2131430.0,203894.0,20905.0,...,22.346818,27.027273,2018-01-19,130.25199,2018-01-19,MAR-VIE,1337.90000,1328.47000,1337.90000,1328.47000
4356,MARTES,2018,1,23,12,1280.86000,1304.30000,2131430.0,203894.0,20905.0,...,20.146364,27.718182,2018-01-23,121.59858,2018-01-23,MAR-VIE,1273.75000,1291.28000,1275.87000,1298.23000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28668,JUEVES,2023,12,7,12,1549.80710,1537.05871,2375346.0,219695.0,25115.0,...,20.952273,27.520909,2023-12-07,810.73198,2023-12-07,MAR-VIE,1565.82985,1542.38100,1565.82985,1542.38100
28692,MARTES,2023,12,12,12,1587.47892,1574.98459,2375346.0,219695.0,25115.0,...,24.013636,30.144545,2023-12-12,920.26224,2023-12-12,MAR-VIE,1569.27978,1552.07366,1549.80710,1537.05871
28716,MIERCOLES,2023,12,13,12,1590.62592,1570.01309,2375346.0,219695.0,25115.0,...,23.340909,29.273636,2023-12-13,927.53024,2023-12-13,MAR-VIE,1587.47892,1574.98459,1587.47892,1574.98459
28740,JUEVES,2023,12,14,12,1603.22201,1588.89158,2375346.0,219695.0,25115.0,...,22.717727,29.004545,2023-12-14,922.66537,2023-12-14,MAR-VIE,1590.62592,1570.01309,1590.62592,1570.01309


In [14]:
fecha_inicio_pred='2024-01-01'
#fecha_fin_pred='2024-03-31'
semanas=13
dias_to_pred=14
fecha_inicio_pred=datetime.strptime(fecha_inicio_pred, "%Y-%m-%d")
fecha_inicio_pred= fecha_inicio_pred - timedelta(days=dias_to_pred//2)
fecha_fin_pred=fecha_inicio_pred + timedelta(days=dias_to_pred-1)




resultados=pd.DataFrame()

for semana in range(semanas):
  inicio = time.time()
  print("pronosticando entre las fechas ",fecha_inicio_pred," y ", fecha_fin_pred)

  #CREACIÓN DE MODELOS
  #-------------------------------------------------------------------------------------------------------------
  #-------------------------------------------------------------------------------------------------------------
  #-------------------------------------------------------------------------------------------------------------
  # Lista de días a considerar
  #dias = ['MAR-VIE','DOM', 'SAB', 'LFEST', 'OFEST', '25-1','MV-DE', 'L-DE', 'LUN-MADLF' ]

  model={}
  # Iterar por cada día y cada hora
  for dia in unique_t_dia:
      for hora in range(24):
          # Filtrar el conjunto de datos por el día y la hora actual
          filtered = filtered_datasets[dia][filtered_datasets[dia]['HORA'] == hora]
          filtered = filtered[(filtered['AÑO'] >= 2018) & (filtered['AÑO'] != 2020)]
          filtered = filtered[filtered['FECHA'] < fecha_inicio_pred.strftime("%Y-%m-%d")]
          # Dividir el dataset en características (X) y variable a predecir (y)
          X = filtered[['Demanda_anterior', 'Delta_24h', 'Delta_25h', 'T2M_pond']]

          #X = filtered[['Demanda_anterior', 'Delta_24h', 'Delta_25h']]

          #X = filtered[['Demanda_anterior', 'Delta_24h', 'Delta_25h', 'Industrial', 'Total Residencial',
          #              'T2M_pond', 'T2M_AREA METROPOLITANA DEL VALLE DE ABURRÁ', 'T2M_MAGDALENA MEDIO',
          #              'T2M_BAJO CAUCA', 'T2M_URABA', 'T2M_ORIENTE',
          #              'Tasa de cambio representativa del mercado (TRM)', 'IPP', 'IPC']]

          y = filtered['DEMANDA']

          # Dividir X y y en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
          #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
          X_train, y_train = X, y

          # Inicializar y ajustar el modelo de regresión lineal
          model[dia,hora] = LinearRegression()
          model[dia,hora].fit(X_train, y_train)
  #-------------------------------------------------------------------------------------------------------------
  #-------------------------------------------------------------------------------------------------------------
  #-------------------------------------------------------------------------------------------------------------


  #PREDICCIONES
  #-------------------------------------------------------------------------------------------------------------
  #-------------------------------------------------------------------------------------------------------------
  #-------------------------------------------------------------------------------------------------------------


  X = final_dataset[(final_dataset['FECHA'] >= fecha_inicio_pred.strftime("%Y-%m-%d")) & (final_dataset['FECHA'] <= fecha_fin_pred.strftime("%Y-%m-%d"))]

  y = X['DEMANDA']
  X = X[['T_DIA', 'HORA','Demanda_anterior', 'Delta_24h', 'Delta_25h', 'T2M_pond']]


  #X = X[['T_DIA', 'HORA','Demanda_anterior', 'Delta_24h', 'Delta_25h',]]
  #X = X[['T_DIA', 'HORA','Demanda_anterior', 'Delta_24h', 'Delta_25h', 'Industrial', 'Total Residencial',
  #              'T2M_pond', 'T2M_AREA METROPOLITANA DEL VALLE DE ABURRÁ', 'T2M_MAGDALENA MEDIO',
  #              'T2M_BAJO CAUCA', 'T2M_URABA', 'T2M_ORIENTE',
  #              'Tasa de cambio representativa del mercado (TRM)', 'IPP', 'IPC']]
  delta24h=X['Delta_24h']
  delta25h=X['Delta_25h']
  T_DIA=X['T_DIA']
  HORA=X['HORA']




  # Lista para almacenar todas las predicciones
  all_y_pred = []


  for indice, fila in X.iterrows():
      dia = fila['T_DIA']
      hora = fila['HORA']

      # Crear un DataFrame con la fila actual
      fila_df = pd.DataFrame(fila).T
      fila_df.drop(['T_DIA', 'HORA'], axis=1, inplace=True)  # Eliminar las columnas 'T_DIA' y 'HORA'

      # Realizar predicción para la fila actual
      y_pred = model[dia, hora].predict(fila_df)

      # Agregar la predicción actual a la lista de todas las predicciones
      all_y_pred.append(y_pred)

      # Buscar el siguiente dia en X donde T_DIA es igual al día actual para luego actualizar la predicción
      next_t_dia = X[(X['T_DIA'] == dia) & (X['HORA'] == hora)  & (X.index > indice)]


      X.at[indice + 1, 'Demanda_anterior'] = y_pred

      if not next_t_dia.empty:
          X_t_dia_first_row_index = next_t_dia.index[0]
          X.at[X_t_dia_first_row_index, 'Delta_24h'] = y_pred
          X.at[X_t_dia_first_row_index, 'Delta_25h'] = fila['Demanda_anterior']



  # crear un nuevo dataframe para comparar predicicones con el real
  all_y_pred = [array[0] for array in all_y_pred]  # Obtener el primer elemento de cada array

  # Crear el DataFrame
  results = pd.DataFrame({'y_test': y, 'y_pred': all_y_pred, '%error': 100*(np.array(all_y_pred) - y) / y})

  results['T_DIA']=T_DIA
  results['HORA']=HORA


  results.reset_index(drop=True, inplace=True)

  fecha_inicio_pred, fecha_fin_pred=fecha_inicio_pred + timedelta(days=dias_to_pred//2), fecha_fin_pred + timedelta(days=dias_to_pred//2)

  resultados= pd.concat([resultados, results[-168:]], ignore_index=True)

  fin = time.time()

  # Calcular la duración de la operación
  duracion = fin - inicio
  print(f"La operación tomó {duracion} segundos.")





pronosticando entre las fechas  2023-12-25 00:00:00  y  2024-01-07 00:00:00
La operación tomó 2.475209951400757 segundos.
pronosticando entre las fechas  2024-01-01 00:00:00  y  2024-01-14 00:00:00
La operación tomó 1.9333891868591309 segundos.
pronosticando entre las fechas  2024-01-08 00:00:00  y  2024-01-21 00:00:00
La operación tomó 1.8239467144012451 segundos.
pronosticando entre las fechas  2024-01-15 00:00:00  y  2024-01-28 00:00:00
La operación tomó 1.7781627178192139 segundos.
pronosticando entre las fechas  2024-01-22 00:00:00  y  2024-02-04 00:00:00
La operación tomó 1.7585184574127197 segundos.
pronosticando entre las fechas  2024-01-29 00:00:00  y  2024-02-11 00:00:00
La operación tomó 1.7932384014129639 segundos.
pronosticando entre las fechas  2024-02-05 00:00:00  y  2024-02-18 00:00:00
La operación tomó 1.9498608112335205 segundos.
pronosticando entre las fechas  2024-02-12 00:00:00  y  2024-02-25 00:00:00
La operación tomó 2.4547436237335205 segundos.
pronosticando ent

In [14]:
resultados,rmse=metricas(resultados['y_test'],resultados['y_pred'])

r2:0.9981011613921519
MSE: 91.21794136216714
RMSE: 9.5508084140646
MAE: 7.060882038327109
MAPE: 0.005724968961692818
shapes: (2184,) (2184,) (2184,)
        y_test       y_pred    %error
0    881.82599   864.088618  2.011437
1    849.93100   835.653498  1.679843
2    809.63944   819.098708  1.168331
3    782.66869   793.420207  1.373700
4    761.60659   782.884040  2.793759
5    762.93479   773.223654  1.348590
6    730.88959   764.840201  4.645108
7    744.82634   795.678558  6.827392
8    779.98440   807.104538  3.477010
9    822.18231   832.860753  1.298793
10   871.63415   868.420060  0.368743
11   922.73952   915.262980  0.810255
12   955.06747   950.385810  0.490192
13   959.40707   952.327587  0.737902
14   947.78873   941.160390  0.699348
15   945.94762   933.138853  1.354067
16   943.25234   933.555968  1.027972
17   956.09995   952.496720  0.376868
18  1051.09203  1056.550861  0.519349
19  1071.34546  1072.157299  0.075778
20  1041.25430  1031.522239  0.934648
21   996.93233 

In [15]:
# Create traces
trace_real = go.Scatter(x=resultados.index, y=resultados['y_test'], mode='lines', name='Demanda Real', line=dict(color='red'))
trace_pred = go.Scatter(x=resultados.index, y=resultados['y_pred'], mode='lines', name='Predicción demanda', line=dict(color='blue'))

# Create layout
layout = go.Layout(title='Predicción demanda horaria usando Regresiones lineales',
                  xaxis=dict(title='Time'),
                  yaxis=dict(title='Demanda'))

# Create figure
fig = go.Figure(data=[trace_real, trace_pred], layout=layout)

# Display interactive plot
fig.show()

In [16]:
# Define el nombre del archivo CSV
nombre_archivo = '/content/drive/MyDrive/Especializacion Analitica y ciencia de datos/Semestre 2/MONOGRAFIA/Pronostico/REGRESION/reg_lin2.csv'


# Guarda el DataFrame en un archivo CSV en la ruta especificada
resultados.to_csv(nombre_archivo, index=False)  # El argumento index=False evita que se guarde el índice del DataFrame